In [1]:
import os
import cv2
import pickle
import torch
from torch.utils.data import DataLoader
device ='cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [2]:
pickle_path = '/purestorage/AILAB/AI_2/youhans/workspace/reid/person/HumanBench/PA100k/dataset_all.pkl'
root_path = "/purestorage/AILAB/AI_2/datasets/ReID/PA-100k/data/release_data/PA-100k/"

In [3]:
pickle_file = pickle.load(open(pickle_path, 'rb'))
print(pickle_file.keys())

image_names = pickle_file['image_name']
labels = pickle_file['label']

dict_keys(['description', 'reorder', 'root', 'image_name', 'label', 'attr_name', 'label_idx', 'partition', 'weight_train', 'weight_trainval'])


In [47]:
import os
import pickle

from PIL import Image
import numpy as np
import torch.utils.data as data
import cv2
# from petrelbox.io import PetrelHelper
from PATH.core.data.transforms.pedattr_transforms import PedAttrAugmentation, PedAttrTestAugmentation, PedAttrRandomAugmentation


class AttrDataset(data.Dataset):

    def __init__(self, ginfo, augmentation, task_spec, train=True, data_use_ratio=1, **kwargs):

        # assert task_spec.dataset in ['peta', 'PA-100k', 'rap', 'rap2', 'uavhuman', 'HARDHC', 'ClothingAttribute', 'parse27k', 'duke', 'market'], \
        #     f'dataset name {task_spec.dataset} is not exist'

        data_path = "/purestorage/AILAB/AI_2/youhans/workspace/reid/person/HumanBench/PA100k/dataset_all.pkl"
        # dataset_info = PetrelHelper.pickle_load(data_path)
        dataset_info = pickle.load(open(data_path, 'rb+'))


        img_id = dataset_info["image_name"]
        attr_label = dataset_info["label"]
        attr_label[attr_label == 2] = 0

        if train:
            split = 'trainval'
        else:
            split = 'test'

        assert split in dataset_info["partition"].keys(), f'split {split} is not exist'

        height = 256
        width = 192

        # self.dataset = "/purestorage/AILAB/AI_2/youhans/workspace/reid/person/HumanBench/PA100k/dataset_all.pkl"
        self.root_path = '/purestorage/AILAB/AI_2/datasets/ReID/PA-100k/data/release_data/PA-100k'

        if train:
            self.transform = PedAttrAugmentation(height, width)
            self.transform = PedAttrRandomAugmentation(height, width, \
                10, 10)
        else:
            self.transform = PedAttrTestAugmentation(height, width)

        self.attr_id = dataset_info["attr_name"]
        self.attr_num = len(self.attr_id)

        self.img_idx = dataset_info["partition"][split]

        # if isinstance(self.img_idx, list):
        #     self.img_idx = self.img_idx[0]  # default partition 0
        #
        # if data_use_ratio != 1:
        #     self.img_idx = random.sample(list(self.img_idx), int(len(self.img_idx) * data_use_ratio))

        # self.img_num = len(self.img_idx)
        self.img_num = self.img_idx.shape[0]
        # self.img_idx = np.array(self.img_idx)

        self.img_id = [img_id[i] for i in self.img_idx]
        self.label = attr_label[self.img_idx]
        # self.task_name = ginfo.task_name


    # def __getitem__(self, index):
    #     return self.read_one(index)

    def __getitem__(self, index):
        print(index)
        imgname = self.img_id[index]
        gt_label = self.label[index]
        img_path = os.path.join(self.root_path, imgname)


        img = Image.open(img_path)
        if self.transform is not None:
            img = self.transform(img)

        gt_label = gt_label.astype(np.float32)
        #
        # img = torch.from_numpy(img).float()
        # gt_label = torch.from_numpy(gt_label).float()
        output = {}
        output = {'image': img, 'label': gt_label, 'filename': imgname}

        return output



    def __len__(self):
        return len(self.img_id)

    # def read_one(self, idx=None):
    #     # if idx == None:
    #     #     idx = np.random.randint(self.img_num)
    #
    #     # imgname, gt_label, imgidx = self.img_id[idx], self.label[idx], self.img_idx[idx]
    #     imgname = self.img_id[idx]
    #     gt_label = self.label[idx]
    #     # imgidx = self.img_idx[idx]
    #     imgpath = os.path.join(self.root_path, imgname)
    #
    #     try:
    #         # img = PetrelHelper.pil_open(imgpath, "RGB")
    #         img = cv2.imread(imgpath)
    #         if self.transform is not None:
    #             img = self.transform(img)
    #
    #         gt_label = gt_label.astype(np.float32)
    #
    #         output = {}
    #         output = {'image': img, 'label': gt_label, 'filename': imgname}
    #         return output
    #     except:
    #         print('{} load failed'.format(imgpath))
    #         return self.read_one()


In [48]:
# total_num = len(image_names)
# num = 0
# type_list = []
# for i, image_name in enumerate(image_names):
#     img_path = os.path.join(root_path, image_name)
#     img = cv2.imread(img_path)
#     label = labels[i]
#     type_list.append(f'img_{type(img)}')
#     type_list.append(f'label_{type(label)}')
#     # if type(img) == None:
#     #     print(image_name)
#     #
#     # if type(label) == None:
#     #     print(image_name)
#
#     num+=1
#
# type_list = list(dict.fromkeys(type_list))
# print(num)
# print(type_list)

dataset = AttrDataset(ginfo=None, augmentation=None, task_spec=None, train=True)
DataLoader = data.DataLoader(dataset, batch_size=16, shuffle=True)

In [49]:
for i, data in enumerate(DataLoader):
    print(data)

14955


AttributeError: 'Tensor' object has no attribute 'transform'